In [9]:
include("../src/Deeplib.jl")
const dlib = Deeplib

LoadError: LoadError: MethodError: no method matching Main.Deeplib.Param(::Int64)

Closest candidates are:
  Main.Deeplib.Param(::T, !Matched::Float64) where T
   @ Main.Deeplib ~/Library/Mobile Documents/iCloud~md~obsidian/Documents/MemorisiCloud/2 Projets/Deeplib/src/AutoDiff.jl:59

in expression starting at /Users/pichedereck/Library/Mobile Documents/iCloud~md~obsidian/Documents/MemorisiCloud/2 Projets/Deeplib/src/AutoDiff.jl:67
in expression starting at /Users/pichedereck/Library/Mobile Documents/iCloud~md~obsidian/Documents/MemorisiCloud/2 Projets/Deeplib/src/Deeplib.jl:1

In [11]:
# Julia can handle object identity in a dictionnary (IdDict)
# this will be used to handle the model parameters and gradients values
d1 = dlib.Dense(3, 3, x->x)
d2 = dlib.Dense(3, 3, x->x)
d3 = dlib.Dense(3, 3, x->x)
println(typeof(d1))

iddict = IdDict(d1=>(d1.w, d1.b), d2=>(d2.w, d2.b))
println(iddict)
println(iddict[d1])
println(iddict[d2])
dict = dlib.GetJacoDict(d1)
println(dict)
println(dict[d1.w])

UndefVarError: UndefVarError: `dlib` not defined

In [12]:
# gives hints on how to pattern match 
f = d1 ∘ d2
println(typeof(f))
f = d1 ∘ d2 ∘ d3
println(typeof(f))

# nesting
function c()
    f = d1 ∘ d2 ∘ d3
    println(typeof(f))
    return f
end
c()
println(typeof(c))

g(x) = d1(d2(x))

# they are not equal. each function has its on type
println(typeof(g))
println( typeof( g ) === typeof( d1 ∘ d2 ) )

# this means that AutoDiff must pattern match on operations, not on 
# high level layers application

UndefVarError: UndefVarError: `d1` not defined

In [13]:

# Testing the forward algorithm for +
h(x) = x
d1 = dlib.Dense(3, 3, h)
d2 = dlib.Dense(3, 3, h)
d3 = dlib.Dense(3, 3, h)

function DSum(x)
    return d1(x) + d2(x) + d3(x) 
end

x = rand(Float16, 3)
println(h(x))
println(d1(x))
println(DSum(x))

dlib.Forward(DSum, x)

UndefVarError: UndefVarError: `dlib` not defined

In [14]:
import Base.+




# This is to show how we can rewrite basic operations like +
function l(a, b)
    return a + b
end

# this structure is only made to pattern match 

struct Param{T}
    v::T
    id::Int64
end

n=2
dictio = IdDict{Any, Any}(n=>2)

# redefine the + operator for Params 
function +(a::Param, b::Param)
    merge!(dictio, IdDict(a=>b))
    #push!(dictio, a=>b)
    return -1
end

function +(a::Param, b::Any)
    merge!(dictio, IdDict(a=>b))
    #push!(dictio, a=>b)
    return -1
end

function +(a::Param, b::Any)
    merge!(dictio, IdDict(a=>b))
    #push!(dictio, a=>b)
    return -2
end

+(b::Any, a::Param) = +(a, b)


function test(f::Function)
    a = rand(Int, 2)
    b = rand(Int, 2)
    println(f(a,b))
    a = Param(a, 340)
    b = Param(b, 450)
    println(f(a,b))
    println(dictio)
    println(dictio[a])
end

test(l)


# now testing partial pattern match!
function test2(f::Function)
    a = rand(Int, 2)
    b = rand(Int, 2)
    println(f(a,b))
    #a = Param(a)
    b = Param(b, 33)
    println(f(a,b))
    println(dictio)
    println(dictio[b])

end

test2(l)


[-4769713272005644257, -8012972000598111837]
-1
IdDict{Any, Any}(2 => 2, Param{Vector{Int64}}([1036328932853735538, 8859413753818621536], 340) => Param{Vector{Int64}}([-5806042204859379795, 1574358319292818243], 450))
Param{Vector{Int64}}([-5806042204859379795, 1574358319292818243], 450)
[-6386184146261228881, -4492546776698002935]
-2
IdDict{Any, Any}(2 => 2, Param{Vector{Int64}}([-3391564524791061973, 8794175845296859208], 33) => [-2994619621470166908, 5160021451714689473], Param{Vector{Int64}}([1036328932853735538, 8859413753818621536], 340) => Param{Vector{Int64}}([-5806042204859379795, 1574358319292818243], 450))
[-2994619621470166908, 5160021451714689473]


In [15]:
# this is to test transitions from param types
dictio = IdDict{Any, Any}()

w1 = Param(2, 300)
w2 = Param(1, 400)

u(x::Int) = x + w1
k(x::Int) = (1 + x) + w2


x = 1
y = u(k(x))

println(dictio)

IdDict{Any, Any}(Param{Int64}(2, 300) => -2, Param{Int64}(1, 400) => 2)


In [16]:
# testing Tag abstract type 

